<a href="https://colab.research.google.com/github/nomnomnonono/FaceRecognizer/blob/main/recognizer/recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from keras.models import Model
from keras.layers import Dense,Input,GlobalMaxPooling2D,Dropout
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [31]:
N_TRAIN = 325
N_VALID = 81
BATCH_SIZE = 8
IMAGE_SIZE = 224
N_CLASS = 3

In [32]:
train_datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    rotation_range = 10,
    horizontal_flip = True
)

valid_datagen = ImageDataGenerator(
    rescale = 1.0 / 255
)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/face/train',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/face/validation',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 325 images belonging to 3 classes.
Found 81 images belonging to 3 classes.


In [33]:
input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(1024, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(1024, activation='relu')(x)
pred = Dense(N_CLASS, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=pred)

for layer in base_model.layers[:15]:
  layer.trainable = False

model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [34]:
cp = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/face/face.hdf5', monitor="val_loss", verbose=1,
                     save_best_only=True, save_weights_only=True)

In [29]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=N_TRAIN//BATCH_SIZE,
                              epochs=15,
                              callbacks=[cp],
                              validation_data=valid_generator,
                              validation_steps=N_VALID//BATCH_SIZE
                              )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
40/40 [==============================] - 13s 324ms/step - loss: 1.1192 - accuracy: 0.4290 - val_loss: 1.0153 - val_accuracy: 0.5250

Epoch 00001: val_loss improved from inf to 1.01530, saving model to /content/drive/MyDrive/Colab Notebooks/face/face.hdf5
Epoch 2/15
40/40 [==============================] - 13s 315ms/step - loss: 0.9004 - accuracy: 0.5773 - val_loss: 0.8577 - val_accuracy: 0.6500

Epoch 00002: val_loss improved from 1.01530 to 0.85775, saving model to /content/drive/MyDrive/Colab Notebooks/face/face.hdf5
Epoch 3/15
40/40 [==============================] - 13s 316ms/step - loss: 0.8304 - accuracy: 0.6562 - val_loss: 0.9132 - val_accuracy: 0.6250

Epoch 00003: val_loss did not improve from 0.85775
Epoch 4/15
40/40 [==============================] - 12s 312ms/step - loss: 0.6520 - accuracy: 0.7035 - val_loss: 0.8177 - val_accuracy: 0.6625

Epoch 00004: val_loss improved from 0.85775 to 0.81775, saving model to /content/drive/MyDrive/Colab Notebooks/face/face.hdf5